In [1]:
%pwd

'c:\\Users\\suraj\\OneDrive\\Desktop\\ML Chatbot\\ML-Chatbot\\research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\suraj\\OneDrive\\Desktop\\ML Chatbot\\ML-Chatbot'

In [4]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
def load_pdf_files(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf_files('data')

In [7]:
extracted_data

[Document(metadata={'producer': 'calibre 3.6.0 [https://calibre-ebook.com]', 'creator': 'calibre 3.6.0 [https://calibre-ebook.com]', 'creationdate': '2018-03-23T06:07:00+00:00', 'author': 'Oliver Theobald', 'title': 'Machine Learning For Absolute Beginners: A Plain English Introduction (Second Edition)', 'source': 'data\\ml-book.pdf', 'total_pages': 128, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'calibre 3.6.0 [https://calibre-ebook.com]', 'creator': 'calibre 3.6.0 [https://calibre-ebook.com]', 'creationdate': '2018-03-23T06:07:00+00:00', 'author': 'Oliver Theobald', 'title': 'Machine Learning For Absolute Beginners: A Plain English Introduction (Second Edition)', 'source': 'data\\ml-book.pdf', 'total_pages': 128, 'page': 1, 'page_label': '2'}, page_content='Machine Learning For Absolute\nBeginners\n \n \n \n \nOliver Theobald'),
 Document(metadata={'producer': 'calibre 3.6.0 [https://calibre-ebook.com]', 'creator': 'calibre 3.6.0 [https://calibre

In [8]:
len(extracted_data)

128

In [9]:
from typing import List
from langchain_core.documents import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    minimal_docs: List[Document] = []
    for doc in docs:
        minimal_doc = Document(
            page_content=doc.page_content,
            metadata={"source": doc.metadata.get("source")}
        )
        minimal_docs.append(minimal_doc)
    return minimal_docs

In [10]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [11]:
minimal_docs

[Document(metadata={'source': 'data\\ml-book.pdf'}, page_content=''),
 Document(metadata={'source': 'data\\ml-book.pdf'}, page_content='Machine Learning For Absolute\nBeginners\n \n \n \n \nOliver Theobald'),
 Document(metadata={'source': 'data\\ml-book.pdf'}, page_content='Second Edition\nCopyright © 2017 by Oliver Theobald\nAll rights reserved. No part of this publication may be reproduced,\ndistributed, or transmitted in any form or by any means, including\nphotocopying, recording, or other electronic or mechanical\nmethods, without the prior written permission of the publisher,\nexcept in the case of brief quotations embodied in critical reviews\nand certain other non-commercial uses permitted by copyright law.'),
 Document(metadata={'source': 'data\\ml-book.pdf'}, page_content='Contents\n \nINTRODUCTION\nWHAT IS MACHINE LEARNING?\nML CATEGORIES\nTHE ML TOOLBOX\nDATA SCRUBBING\nSETTING UP YOUR DATA\nREGRESSION ANALYSIS\nCLUSTERING\nBIAS & VARIANCE\nARTIFICIAL NEURAL NETWORKS\nDECIS

In [12]:
# Chunking

def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [13]:
texts_chunk = text_split(minimal_docs)

print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 410


In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [3]:
embeddings = download_embeddings()

C:\Users\suraj\AppData\Local\Temp\ipykernel_19448\688634661.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\suraj\miniconda3\envs\mlbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\suraj\miniconda3\envs\mlbot\lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not

In [4]:
embeddings


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [5]:
vector = embeddings.embed_query("What is the capital of France?")
vector

[0.08204811811447144,
 0.036055516451597214,
 -0.003892872715368867,
 -0.004881036933511496,
 0.025651132687926292,
 -0.05714348703622818,
 0.012191579677164555,
 0.004678918048739433,
 0.034949880093336105,
 -0.022421924397349358,
 -0.008005263283848763,
 -0.10935354232788086,
 0.02272474206984043,
 -0.029320882633328438,
 -0.04352206736803055,
 -0.12024123221635818,
 -0.0008486550068482757,
 -0.018150104209780693,
 0.056129515171051025,
 0.003085274016484618,
 0.0023363486398011446,
 -0.016839247196912766,
 0.06362469494342804,
 -0.02366018481552601,
 0.03149354085326195,
 -0.034797921776771545,
 -0.02054883912205696,
 -0.002790941158309579,
 -0.011037981137633324,
 -0.036126721650362015,
 0.0541410855948925,
 -0.03661714121699333,
 -0.02500864677131176,
 -0.03817039728164673,
 -0.04960361868143082,
 -0.015148100443184376,
 0.021315040066838264,
 -0.01274044718593359,
 0.07670091092586517,
 0.04435574635863304,
 -0.01083483174443245,
 -0.02976001612842083,
 -0.01697045937180519,
 -0.

In [6]:
print(len(vector))

384


In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [5]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [6]:
pc

In [10]:
# Database
from pinecone import ServerlessSpec

index_name = "machine-learning-chatbot"
if not pc.has_index(index_name):
    pc.create_index(name=index_name, dimension=384, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"))

index = pc.Index(index_name)

c:\Users\suraj\miniconda3\envs\mlbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)